In [ ]:
# Вставить id приложения сюда
your_id = ""  # {type:"string"}
print(
    f"Cсылка: "\
    "https://oauth.vk.com/authorize?client_id={your_id}"\
    "&display=page&redirect_uri=https://oauth.vk.com/blank.html"\
    "&scope=offline&response_type=token&v=5.131"
)

In [ ]:
# Вставить ссылку сюда:
link = "https://id.vk.com/auth?return_auth_hash=0de8e14f2bebdc4e1d&redirect_uri=https://oauth.vk.com/blank.html&redirect_uri_hash=4b8f2bd2015bc72c67&force_hash=1&app_id=51778462&response_type=token&code_challenge=&code_challenge_method=&scope=65536&state=" # {type:"string"}
token = '' # Вставить сюда токен


In [ ]:
#Парсер комментариев

import requests
import pandas as pd
from tqdm import tqdm
import datetime
import time

def unix_to_date(ts): #перевод даты из unix-формата в привычный
    return datetime.datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

def nes_params(post_id, all_comments): #работа с необходимыми параметрами.
    nes_dict = {}
    profiles = all_comments['profiles']
    comments = all_comments['items']
    first_string = ['NONE', 'NONE', 'NONE']
    for comment in comments:
        if len(comment['text']) > 3:
            second_string = [unix_to_date(comment['date']), comment['likes']['count'],
                             comment['text']]
            for profile in profiles:
                if comment['from_id'] == profile['id']:
                    first_string = [profile['first_name'], profile['last_name']]
            nes_dict[comment['id']] = first_string + second_string + [i]
    return nes_dict

def get_Comments(post_id, owner_id, nes_dict, token): #парсинг комментариев по id постов
    version = 5.131
    offset = 0
    count = 100
    while offset < 500:
        response = requests.get('https://api.vk.com/method/wall.getComments',
                                params={
                                    'access_token': token,
                                    'v': version,
                                    'owner_id': owner_id,
                                    'post_id': post_id,
                                    'need_likes': 1,
                                    'count': count,
                                    'offset': offset,
                                    'extended': 1
                                }
                                )
        data_comments = response.json()['response']
        tempDict = nes_params(post_id, data_comments)
        nes_dict.update(tempDict)
        offset += 100
        time.sleep(0.5)
    return nes_dict

def to_df(nes_dict): #перевод словаря в датафрейм
    df = pd.DataFrame.from_dict(nes_dict, orient='index',
                                columns=['name', 'last_name', 'date', 'likes', 'text', 'post_id'])
    return df

owner_id =  -51988192 #{type:"number"} ID группы
post_id = "..." # {type:"string"}

nes_dict = {}
for i in tqdm(post_id.split()):
   get_Comments(i, owner_id, nes_dict, token)
df = to_df(nes_dict)
# df.to_csv('/content/' + f'{owner_id}.csv', sep=';', index=False)
# print(f'\n Таблица сохранилась на вашем диске под названием {owner_id}.csv')